In [ ]:
"""
Purpose: To Run the neuron preprocessing


"""

In [ ]:
from os import sys
sys.path.append("/meshAfterParty/")
from importlib import reload

import datajoint as dj
from pathlib import Path

import datajoint_utils as du
du = reload(du)
from importlib import reload

import time

# configuring the virtual module

In [ ]:
import minfig
du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

# Defining Our Table

In [ ]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [ ]:
#so that it will have the adapter defined
from datajoint_utils import *

In [ ]:
import numpy as np
@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation
    ---
    decomposition: <decomposition>
    n_error_limbs: int #the number of branches that are touching multiple somas or 1 soma in multiple places
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches:int
    
    skeletal_length: double
    max_limb_skeletal_length:double
    median_branch_length:double #gives information on average skeletal length to next branch point
    
    
    width_median: double #median width from mesh center without spines removed
    width_no_spine_median: double #median width from mesh center with spines removed
    width_90_perc: double # 90th percentile for width without spines removed
    width_no_spine_90_perc: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density: double # n_spines/ skeletal_length
    spines_per_branch: double
    
    skeletal_length_eligible: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches; int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume: double # the sum of all spine volume
    spine_volume_density: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible: double: #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible: double #total_spine_volume/n_spine_eligible_branche

    
    """

    key_source = minnie.Decimation() & (minnie.BaylorSegmentCentroid() & "multiplicity>0").proj()

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = key['version']
        print(f"\n\n----- Working on {segment_id}-------")

        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)

        #3) Get the somas info *************************** Need to change this when actually run *******************
        #somas = du.get_soma_mesh_list(segment_id) 
        somas = None

        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"]

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Calculate all starter stats
        # ----------- calculating the attributes --------- #
        n_error_limbs = len(nru.error_limb_indexes(recovered_neuron))

        n_somas = len(recovered_neuron.get_soma_node_names())

        n_limbs = len(recovered_neuron.get_limb_node_names())

        n_branches_per_limb = [len(ex_limb.get_branch_names()) for ex_limb in recovered_neuron]
        n_branches = np.sum(n_branches_per_limb)

        n_spines = len(recovered_neuron.spines)

        #for total skeletal length
        sk_len_per_limb = [sk.calculate_skeleton_distance(limb.skeleton) for limb in recovered_neuron]
        skeletal_length = np.sum(sk_len_per_limb)

        if skeletal_length > 0:
            spine_density = n_spines/skeletal_length
        else:
            spine_density = 0

        max_limb_skeletal_length = np.max(sk_len_per_limb)

        max_limb_n_branches = np.max(n_branches_per_limb)
        
        
        
        
        #5b) --------------- Compute the stats suggested by Jake -------------------------
        
        
        # --> preprocessing
        all_skeletal_lengths = []
        all_widths_no_spine = []
        all_widths = []

        n_spine_eligible_branches = 0
        spine_eligible_branch_lengths = []

        for curr_limb in recovered_neuron:
            for curr_branch in curr_limb:
                curr_branch_sk_len = sk.calculate_skeleton_distance(curr_branch.skeleton)
                all_skeletal_lengths.append(curr_branch_sk_len)
                all_widths_no_spine.append(curr_branch.width_new["no_spine_median_mesh_center"])
                all_widths.append(curr_branch.width_new["median_mesh_center"])

                if not curr_branch.spines is None:
                    spine_eligible_branch_lengths.append(curr_branch_sk_len)
                    n_spine_eligible_branches += 1

        all_skeletal_lengths = np.array(all_skeletal_lengths)
        median_branch_length = np.round(np.median(all_skeletal_lengths),3)
        
        
        #--> width data
        width_median = np.round(np.median(all_widths),3)
        width_no_spine_median = np.round(np.median(all_widths_no_spine),3)

        width_90_perc = np.round(np.percentile(all_widths,90),3)
        width_no_spine_90_perc = np.round(np.percentile(all_widths_no_spine,90),3)
        
        
        
        # -->  spine data
        if n_branches > 0:
            spines_per_branch = np.round(n_spines/n_branches,3)
        else:
            spines_per_branch = 0
        
        #spine eligible density and per branch
        skeletal_length_eligible = np.sum(spine_eligible_branch_lengths)
        if skeletal_length_eligible > 0:
            spine_density_eligible = n_spines/skeletal_length_eligible
        else:
            spine_density_eligible = 0

        if n_branches > 0:
            spines_per_branch_eligible = np.round(n_spines/n_spine_eligible_branches,3)
        else:
            spines_per_branch_eligible = 0

        skeletal_length_eligible,n_spine_eligible_branches,spine_density_eligible,spines_per_branch_eligible
        
        
        spine_time = time.time()
        total_spines = recovered_neuron.spines
        total_volume =[]
        for ts in total_spines:
            vol = tu.mesh_volume(total_spines[0],
                                     watertight_method=None,
                                     return_closed_mesh=False,
                             verbose=False)
            total_volume.append(vol)

        print(f"Total time for calculating spine volume = {time.time() - spine_time}")
        total_volume = np.array(total_volume)

        spine_volume_median = np.median(total_volume)

        total_spine_volume = np.sum(total_volume)

        if skeletal_length > 0:
            spine_volume_density = total_spine_volume/skeletal_length
        else:
            spine_volume_density = 0


        if skeletal_length_eligible > 0:
            spine_volume_density_eligible = total_spine_volume/skeletal_length_eligible
        else:
            spine_volume_density_eligible = 0

        if n_branches > 0:
            spine_volume_per_branch_eligible = np.round(total_spine_volume/n_spine_eligible_branches,3)
        else:
            spine_volume_per_branch_eligible = 0


        

        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")

        
        """
        n_error_limbs: int #the number of branches that are touching multiple somas or 1 soma in multiple places
        n_somas: int #number of soma meshes detected
        n_limbs: int
        n_branches: int
        max_limb_n_branches:int

        skeletal_length: double
        max_limb_skeletal_length:double
        median_branch_length:double #gives information on average skeletal length to next branch point


        width_median: double #median width from mesh center without spines removed
        width_no_spine_median: double #median width from mesh center with spines removed
        width_90_perc: double # 90th percentile for width without spines removed
        width_no_spine_90_perc: double  # 90th percentile for width with spines removed


        n_spines: bigint

        spine_density: double # n_spines/ skeletal_length
        spines_per_branch: double

        skeletal_length_eligible: double # the skeletal length for all branches searched for spines
        n_spine_eligible_branches; int # the number of branches that were checked for spines because passed width threshold

        spine_density_eligible:double # n_spines/skeletal_length_eligible
        spines_per_branch_eligible:double # n_spines/n_spine_eligible_branches

        total_spine_volume: double # the sum of all spine volume
        spine_volume_density: double #total_spine_volume/skeletal_length
        spine_volume_density_eligible: double: #total_spine_volume/skeletal_length_eligible
        spine_volume_per_branch_eligible: double #total_spine_volume/n_spine_eligible_branche
        
        """
        
        

        #7) Pass stats and file location to insert
        new_key = dict(key,
                       decomposition=ret_file_path_str,
                        n_error_limbs=n_error_limbs,
                        n_somas=n_somas,
                        n_limbs=n_limbs,
                        n_branches=n_branches,
                        max_limb_n_branches=max_limb_n_branches,
                       
                        skeletal_length=skeletal_length,
                        max_limb_skeletal_length=max_limb_skeletal_length,
                        median_branch_length=median_branch_length,

                        width_median=width_median, #median width from mesh center without spines removed
                        width_no_spine_median=width_no_spine_median, #median width from mesh center with spines removed
                        width_90_perc=width_90_perc, # 90th percentile for width without spines removed
                        width_no_spine_90_perc=width_no_spine_90_perc,  # 90th percentile for width with spines removed

                        n_spines=n_spines,

                        spine_density=spine_density, # n_spines/ skeletal_length
                        spines_per_branch=spines_per_branch,

                        skeletal_length_eligible=skeletal_length_eligible, # the skeletal length for all branches searched for spines
                        n_spine_eligible_branches=n_spine_eligible_branches,

                        total_spine_volume=total_spine_volume, # the sum of all spine volume
                        spine_volume_density=spine_volume_density, #total_spine_volume/skeletal_length
                        spine_volume_density_eligible=spine_volume_density_eligible, #total_spine_volume/skeletal_length_eligible
                        spine_volume_per_branch_eligible=spine_volume_per_branch_eligible, #total_spine_volume/n_spine_eligible_branche

                       
                      )
#         else:
#             ret_file_path_str = "/mnt/dj-stor01/platinum/minnie65/02/decomposition/864691135574982852_0.pbz2"
#             new_key = dict(key,
#                            decomposition=ret_file_path_str,
#                             n_error_limbs=0,
#                             n_somas=0,
#                             n_limbs=0,
#                             n_branches=0,
#                             n_spines=0,
#                             skeletal_length=0,
#                             spine_density=0,
#                             max_limb_skeletal_length=0,
#                             max_limb_n_branches=0
#                           )
        
        
        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

    

In [ ]:
minnie.Decimation & (minnie.BaylorSegmentCentroid() & "multiplicity>=2").proj()

In [ ]:
((schema.jobs & "table_name = '__decomposition'") & "timestamp>'2020-11-16 00:26:00'").delete()


In [ ]:
# errored_keys = (schema.jobs & "table_name = '__decomposition'").fetch("key")#.delete()
# errored_keys

In [ ]:
import random
import compartment_utils as cu
cu = reload(cu)
import preprocessing_vp2 as pre
pre = reload(pre)
import trimesh as tu
tu = reload(tu)

Decomposition.populate(reserve_jobs=True, suppress_errors=False, order='random')

In [ ]:
debug

In [ ]:
adapter_objects["decomposition"]

In [ ]:
"""
debugging the insert
saved path: /mnt/dj-stor01/platinum/minnie65/02/decomposition/864691135574982852_0.pbz2
self.spec['stage'] did not have a stage key
"""

In [ ]:
import system_utils as su

In [ ]:
import trimesh_utils as tu
main_mesh = tu.load_mesh_no_processing("main_mesh")
current_soma = tu.load_mesh_no_processing("current_soma")

In [ ]:
import neuron_visualizations as nviz

In [ ]:
nviz.plot_objects(meshes=[main_mesh,current_soma],
                 meshes_colors="random")

In [ ]:
stitching_mesh = su.decompress_pickle("stitching_mesh.pbz2")

In [ ]:
import trimesh_utils as tu
tu.split(stitching_mesh)

In [ ]:
import neuron_visualizations as nviz
nviz.plot_objects(submesh)